In [ ]:
# 影片偵測
import cv2
from ultralytics import YOLO
model = YOLO("yolov8n-pose.pt")

path = "./fall.mp4"
cap = cv2.VideoCapture(path)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform pose detection
    results = model.track(frame) #偵測影片中人體關節點.track 可以將偵測到的人加上id
    annotated_frame = results[0].plot() #.plot 將關鍵點畫到影片上
    r = results[0] # r = 輸出結果 
    #印出 所有關鍵點的座標位置
    print(r.keypoints.xyn.numpy().reshape(-1, 2)) 
    #印出 所有關鍵點的信心值

    #將所有關鍵點的位置與信心值丟入knn/svm/random forest/mlp 模型進行預測(0:沒跌倒/1:跌倒)

    # 若是跌倒輸出圖片,並註記時間

    # 存進database

    #通知line，並將圖片送出
    cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

In [ ]:
import csv
from ultralytics import YOLO
# Load YOLO model
image_path = "C:/Users/User/Desktop/Fall Detection/detect/detected_9281.jpg"
model = YOLO("yolov8n-pose.pt")
results = model(image_path)
r = results[0] 
# by 照片人數去抓每個人的keypoints 
# 照片人數 = len(results[0].boxes)
for i in range(len(results[0].boxes)):
    # print(r.boxes.xywhn[i])
    print(r.keypoints.conf[i]) # keypoint共有17個點,每個點yolo偵測到會給一個信心值
    print(r.keypoints.xyn[i]) # keypoint共有17個點,yolo偵測到的x,y座標並做標準化
    print("-----------------")

# Initialize a list to store the combined (x, y, confidence) tuples
combined_results = []

# Iterate over each keypoint
for i in range(len(r.keypoints.xyn)):
    for  j  in range(17):
        x, y = r.keypoints.xyn[i][j].numpy() # Convert tensor to numpy
        confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
        combined_results.append((x, y, confidence))
print(combined_results)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 209):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/fall{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"fall{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 167):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/not fallen{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"not fallen{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os

directory = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"

missing_files = []

for i in range(1, 209):
    filename = os.path.join(directory, f"fall{i:03d}.jpg")
    if not os.path.exists(filename):
        missing_files.append(i)

if missing_files:
    print("Missing files:")
    print(missing_files)
else:
    print("All files exist.")
